# Ping Tutorial
<i>Adapted for use with FABRIC from [Layer2Ping](https://groups.geni.net/geni/wiki/Tutorials/Layer2Ping/Procedure)  and [Lab Zero: A First Experiment Using GENI](https://groups.geni.net/geni/wiki/GENIExperimenter/Tutorials/jacks/GettingStarted_PartI/Procedure).
    
<img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Ping/figures/2vmvlan_overview.png?raw=true" ><br>
In this tutorial you will learn:
* How to create a FABRIC slice and connect to nodes using SSH
* How to create a Layer 2 connection between nodes
    
<b> Prerequisites:  
    
You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
<br>This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)
   
<b>Description of the experiment:    
    
This assignment serves as an introduction to networking concepts. It begins by analyzing existing output from Linux commands, leveraging this knowledge to complete an implementation of the PingPlus program, fashioned after Linux's 'ping' command.
<br>The assignment is structured into two parts:
 - The first phase of this experiment is composed of an introduction to the network interface, ping, and iperf commands in Linux. we use these commands to check the connectivity of the nodes in the network by looking at their useful metrics such as RTT and bandwidth.
 - In the second phase of the assignment we will use the PingPlus program to take a look at an implementation of the ping command on Linux might be executed, In this phase, you will attempt to implement the RTT feature of the ping command. PingPlus uses the MAC address  and the interface name, instead of the IP address of the node to form a connection, allowing us to write a program that functions similarly to the 'ping' command.

## 1. Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.
#### Import the Fabric API

In [ ]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                    
fablib.show_config()
import json
import traceback

try:
    slice_name = 'MyLayer2PingSlice'
    
    slice = fablib.get_slice(slice_name)
    slice.list_nodes()
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
except Exception as e:
    print(f"Get Slices Fail: {e}")

### 1.2 View Topology

#### Get the topology
To configure the Nodes we need to get the topology of the experiment by getting the slice component, in this tutorial we get the slice by the name given when the slice was created.
The folowing cell prints the topology.

In [ ]:
# Preview Slice node's attributes
try:
    slice = fablib.get_slice(slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   Interfaces        : {node.get_interfaces()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print()                
except Exception as e:
    print(f"Fail: {e}")

## 2. Guided experiment

From here on you will be working directly from the terminal.
### 2.1 Open terminal windows
Open two terminal windows by clicking the blue "+" to open the launcher, selecting "Terminal," and repeating.
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Ping/figures/AddTerminal.png?raw=true" ><br>
### 2.2 SSH to nodes
Use the ssh commands from the "1.2 get topology" code block to login to the server in one terminal and the client in the other terminal.
### 2.3 Check IP address
In each terminal, type `ip addr show`. You should see at least two interfaces:
* The control interface. This is the interface you use to access the node, e.g. by ssh. The control interface is mainly used for control traffic (likely something like 10.20.4.18 on enp3s0).
 * The data interface. This is the interface that is used for sending experimental traffic and connects to the other nodes of your experiment through FABRIC. The links between these interfaces are the ones that allow you to run non-IP experiments. The data interface is the one that has an IP address and mask that match what you configured before you reserved your resources (10.20.30.40 for the client and 10.20.30.41 for the server, both on enp7s0).   
 
Write down the ip address and name for the control and data interfaces on each node.  

### 2.4 Verify you can connect between nodes
<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Ping/figures/PingTop.png?raw=true" ><br>
We will use ping to do this.  
    In the server terminal, type `ping -c 3 10.20.30.40`  
    In the client terminal, type `ping -c 3 10.20.30.41`  
If the pings succeed you know you have IP connectivity between the two nodes.  

### 2.5 Install necessary software
Run the below commands on both nodes
<br> `sudo apt-get update`
<br> `sudo apt-get install iperf`
<br> `sudo apt-get install gcc -y`
<br> `sudo apt-get install make`


### 2.6 Measure the bandwidth

1. Start an iperf server on the server node:
<br>`iperf -s`


2. Run an iperf client via the data interface:
<br>`iperf -c <server data IP addr> ` 
    
    
* What is the bandwidth of this link? Why?  

3. Run an iperf client via the control plane:
<br>`iperf -c <server control IP addr>`

* What is the bandwidth of this link? Why?  

4. Type `Ctrl+C` on the server node to stop the iperf server. 

### 2.7 Download and install PingPlus on the server and client nodes
We will download and install a program that does Layer 2 pings on the server and client nodes. Do this on both nodes:  

<br>`wget https://github.com/fabric-testbed/teaching-materials/raw/main/Aditional%20Materials/PingPlus/pingPlus-0.2.tar.gz`
<br>`tar xvfz pingPlus-0.2.tar.gz`

<br>`wget https://github.com/fabric-testbed/teaching-materials/raw/main/Aditional%20Materials/PingPlus/pingPlus-0.2.1.tar.gz`
<br>`tar xvfz pingPlus-0.2.1.tar.gz`

You should see a directory called 'home' navigate through this directory until you see 'pingPlus-0.2'. This directory has the source code for pingPlus. 
### 2.8 Compile pingPlus
Compile pingPlus on both nodes:
<br>`cd pingPlus-0.2`
<br>`make`
    
### 2.9 Turn off IP on the data interface
We now turn off IP on the data interface. Be very careful to not turn off IP on any of the other interfaces as this will make your node unreachable.
<br>`sudo apt install net-tools`
<br>`sudo /sbin/ifconfig <data_interface_name> 0.0.0.0 `
    
### 2.10 Run pingPlus 

<br><img src="https://github.com/fabric-testbed/teaching-materials/blob/main/assignments/Ping/figures/pingPlus.png?raw=true"><br>

In the server terminal, run:
<br>`sudo ./pingPlusListener 10002`

The 10002 is the value we'll use for the type field in the header of the Ethernet frames we send and receive. The above command tells the server to look for Ethernet packets of this type. (In this exercise you can use almost any value for this type field.)

In the client terinal, run:

`sudo ./pingPlus <server_data_interface_hardware_addr> <client_data_interface_name> 10002`

*the `<server_data_interface_hardware_addr>` is the mac adress, if you are following this tutorial, run ifconfig on the server node, it will be the **ether** field on the correct data interface*

This tells the client to send an Ethernet frame with destination address set to the server's data interface MAC address and with the type field set to 10002. This frame will be sent out of the specified interface on the client. The client prints out the contents of the data field of the frame it sent to the server and the contents of the data field in the frame it got back from the server.

In the server terminal, kill the pingPlusListener by typing Ctrl+C.
    
### 2.11 Examine the pingPlus program

Open up an editor (vi or emacs) to view pingPlus.c. All the interesting stuff happens starting line 110 of this file:

* Open a Layer 2 socket (lines 110 - 115)
* Bind the socket to the specified interface (lines 117 - 125)
* In a buffer compose the contents of the data portion of the Ethernet frame to be sent to the server. In our case the data portion has the format "RQ:<randomNumber>+<randomNumber>" (line 134)
* Send the packet. The send is successful if the return from sendPacket matches the number of bytes sent. The sendPacket function is implemented in the file packetFunctions.c in this directory. This function sets the appropriate header fields in the ethernet frame before sending it.
* Wait for the server to respond (line 145). The receivePacket function is implemented in file packetFunctions.c. 

### 2.12 Modify pingPlus to measure round-trip delays

You should by now have a feel for how Layer 2 sockets work. You can now modify ping2Plus to measure round-trip delays similar to the Layer 3 ping. You will want to:

* get the current time before you send the packet using
* get the current time when you get a response from the server. 

### 2.13 (Optional) Get a solution to the exercise

To see a solution to this exercise, create a new directory and dowload the solution to that directory:

<br>`cd `
<br>`wget https://github.com/fabric-testbed/teaching-materials/raw/main/Aditional%20Materials/PingPlus/pingPlus_v3.tar.gz`   
<br>`tar xvfz pingPlus_v3.tar.gz`
<br>`cd pingPlus_v3`

The file pingPlus.c in this directory will have the solution. The statements that do the timing measurement are around lines 151 and 175.


## Cleanup Resources

In [ ]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = False
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")